In [ ]:
from PIL import Image
import matplotlib.pyplot as plt
import numpy as np
from PIL import ImageFilter
import math

# Load the image and convert to Grayscale (L) for clearer understanding of intensity transformations
original_img = Image.open("heart.png").convert('L')
print("Image loaded.")

def show_comparison(img_original, img_transformed, title_transformed):
    plt.figure(figsize=(15, 5))
    
    # Original
    plt.subplot(1, 3, 2)
    plt.imshow(img_original, cmap='gray', vmin=0, vmax=255)
    plt.title("Original Image")
    plt.axis('off')
    
    # Transformed
    plt.subplot(1, 3, 3)
    plt.imshow(img_transformed, cmap='gray', vmin=0, vmax=255)
    plt.title(title_transformed)
    plt.axis('off')
    
    plt.show()

def equalize_histogram(img):
    histogram = img.histogram()
    total_pixels = sum(histogram)

    # 2. Calculate the Cumulative Distribution Function (CDF) and create LUT
    # We perform a cumulative sum and normalize it immediately to 0-255
    equalization_lut = []
    cumulative_sum = 0

    for count in histogram:
        cumulative_sum += count
        s = (cumulative_sum / total_pixels) * 255
        equalization_lut.append(int(s))

    # 3. Apply the LUT
    equalized_img = img.point(equalization_lut)
    return equalized_img

In [ ]:
# --- PARAMETERS TO PLAY WITH ---
size = (3, 3) # Grid dimensions (width, height). Try (5, 5) for stronger blur.

# A Box filter uses all 1s.
# We must create a list with length = width * height.
weights = [1] * (size[0] * size[1]) 

# We must divide the sum by the number of pixels to average them (Normalization).
scale = sum(weights) 
# -------------------------------

# 1. Define the Kernel
# size: tuple of dimensions
# kernel: flat list of weights
# scale: divides the result (essential for averaging!)
# offset: adds to the result (usually 0)
box_kernel = ImageFilter.Kernel(size, weights, scale=scale, offset=0)

# 2. Apply the Kernel
interations = 100  # Number of times to apply the filter
box_blurred_img = original_img.filter(box_kernel)
for _ in range(interations):
    box_blurred_img = box_blurred_img.filter(box_kernel)

# 3. Visualize
show_comparison(original_img, box_blurred_img, f"Box Blur {size}")

In [ ]:
# --- PARAMETERS TO PLAY WITH ---
# This is a fixed 3x3 Gaussian approximation. 
# Try changing the center '4' to '8' (and update scale total) -> less blur.
# Try changing the center '4' to '1' -> heavy blur.
# -------------------------------

# 1. Define the weights
# Note: The sum of these weights is 1 + 2 + 1 + 2 + 4 + 2 + 1 + 2 + 1 = 16
gaussian_weights = [
    1, 2, 1,
    2, 4, 2,
    1, 2, 1
]

# We divide by the sum (16) to maintain image brightness.
scale = sum(gaussian_weights)

# 2. Define the Kernel
gaussian_kernel = ImageFilter.Kernel((3, 3), gaussian_weights, scale=scale)

# 3. Apply the Kernel
interations = 100  # Number of times to apply the filter
gaussian_img = original_img.filter(gaussian_kernel)
for _ in range(interations):
    gaussian_img = gaussian_img.filter(gaussian_kernel)

# 4. Visualize
show_comparison(original_img, gaussian_img, "Gaussian Blur (3x3)")

In [ ]:
# use the gaussian image from here onward
#original_img = gaussian_img

In [ ]:
# --- FIRST ORDER DERIVATIVE KERNELS ---
# Sobel is standard, but you can try the "Prewitt" filter by changing the '2's to '1's.
# -------------------------------

# 1a. Define the raw gradient Filter Kernels 
# Vertical Edge Detector (Responds to Left/Right changes)
w_x_weights = [
    0, 0, 0,
    -0.5, 0, 0.5,
    0, 0, 0
]
# Horizontal Edge Detector (Responds to Top/Bottom changes)
w_y_weights = [
    0, -0.5, 0,
     0,  0,  0,
     0,  0.5, 0
]

"""
# 1b. Define the Sobel Filter Kernels 
# Vertical Edge Detector (Responds to Left/Right changes)
w_x_weights = [
    -1, 0, 1,
    -2, 0, 2,
    -1, 0, 1
]
# Horizontal Edge Detector (Responds to Top/Bottom changes)
w_y_weights = [
    -1, -2, -1,
     0,  0,  0,
     1,  2, 1
]
#"""

# 2. Apply Filters separately
# We use Pillow to filter, but we convert to 'I' (32-bit Integer) mode 
# so we don't lose negative values (gradients can be negative!)
img_i = original_img.convert('I') # Allows negative numbers

Gx_img = img_i.filter(ImageFilter.Kernel((3, 3), w_x_weights, scale=1))
Gy_img = img_i.filter(ImageFilter.Kernel((3, 3), w_y_weights, scale=1))

# 3. Combine them (The Gradient Magnitude)
# We switch to Numpy for the math: sqrt(Gx^2 + Gy^2)
Gx = np.array(Gx_img)
Gy = np.array(Gy_img)

# Calculate magnitude
magnitude = np.sqrt(Gx**2 + Gy**2)
#magnitude = np.abs(Gx) + np.abs(Gy)

# Normalize to 0-255 for display
magnitude = (magnitude / magnitude.max()) * 255
sobel_filtered_image = Image.fromarray(magnitude.astype(np.uint8))

# 4. Visualize
plt.figure(figsize=(20, 10))
plt.subplot(1, 4, 1); plt.imshow(original_img, cmap='gray'); plt.title("Original image)"); plt.axis('off')
plt.subplot(1, 4, 2); plt.imshow(Gx, cmap='gray'); plt.title("Gradient X (Vertical Edges)"); plt.axis('off')
plt.subplot(1, 4, 3); plt.imshow(Gy, cmap='gray'); plt.title("Gradient Y (Horizontal Edges)"); plt.axis('off')
plt.subplot(1, 4, 4); plt.imshow(sobel_filtered_image, cmap='gray'); plt.title("Combined Gradient"); plt.axis('off')
plt.tight_layout()
plt.show()

In [ ]:
# --- SECOND ORDER DERIVATIVE KERNELS ---

# --- PARAMETERS TO PLAY WITH ---
# The center value controls the sharpness intensity.
# Try center = 5 (Standard sharp)
# Try center = 4 (Edge detection only - image becomes mostly black)
center_val = 4
# -------------------------------

# 1. Define the weights
# Standard Laplacian Sharpening (4-neighbor)
laplacian_weights = [
     0,         -1,          0,
    -1,  center_val,        -1,
     0,         -1,          0
]

# If center_val is 5 and neighbors sum to -4, scale is 1.
scale = sum(laplacian_weights) 

# Safety check: if parameters create a sum of 0, we avoid division by zero
if scale == 0: scale = 1 

# 2. Define Kernel
sharpen_kernel = ImageFilter.Kernel((3, 3), laplacian_weights, scale=scale)

# 3. Apply Sharpening
sharpened_img = original_img.filter(sharpen_kernel)

# 4. Visualize
show_comparison(original_img, sharpened_img, "Laplacian Sharpening")
if center_val == 4:
    print("Applying Histogram Equalization...")
    equalized_img = equalize_histogram(sharpened_img)
    show_comparison(sharpened_img, equalized_img, "Image enhancement")